In [1]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

from marketprediction import config
from marketprediction import utils
from marketprediction import plots

In [2]:
tickers = ['AAPL', 'MSFT']
dfs = {}
for ticker in tickers:
    dfs[ticker] = (
        pd.read_excel(config.DATA_FILE, ticker)
        .set_index('Date')
        .rename(columns={f"{metric}": f"{metric} {ticker}" for metric in ['Close', 'High', 'Low', 'Open', 'Volume']})
    )
data = dfs[tickers[0]]
for ticker in tickers[1:]:
    data = data.join(dfs[ticker])

In [ ]:
metric = 'Close'
ticker = 'AAPL'

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data[f'{metric} {ticker}']))
fig.update_layout(title=f'{metric} {ticker}')
fig.show()

In [ ]:
strategy = utils.MovingAverageStrategy()
bt = utils.Backtester(tickers, data, strategy, initial_capital=50)
bt.run()
wallet = bt.wallet

plots.plot_return(wallet, data, tickers)